In [1]:
#read the dataset
import pandas as pd
df = pd.read_csv("C:/Users/sriva/Downloads/bc_trip259172515_230215.csv")
print(df.head()) 

   EVENT_NO_TRIP  EVENT_NO_STOP            OPD_DATE  VEHICLE_ID  METERS  \
0      259172515      259172517  15FEB2023:00:00:00        4223      40   
1      259172515      259172517  15FEB2023:00:00:00        4223      48   
2      259172515      259172517  15FEB2023:00:00:00        4223      57   
3      259172515      259172517  15FEB2023:00:00:00        4223      73   
4      259172515      259172517  15FEB2023:00:00:00        4223     112   

   ACT_TIME  GPS_LONGITUDE  GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP  
0     20469    -122.648137     45.493082              12       0.7  
1     20474    -122.648240     45.493070              12       0.8  
2     20479    -122.648352     45.493123              12       0.8  
3     20484    -122.648385     45.493262              12       0.7  
4     20489    -122.648347     45.493582              12       0.8  


C. Filtering

In [2]:
#filtering using drop
df.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis=1)

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582
...,...,...,...,...,...,...,...
156,259172515,15FEB2023:00:00:00,4223,5834,21389,-122.677057,45.528040
157,259172515,15FEB2023:00:00:00,4223,5838,21424,-122.677003,45.528037
158,259172515,15FEB2023:00:00:00,4223,5858,21429,-122.676765,45.528043
159,259172515,15FEB2023:00:00:00,4223,5889,21434,-122.676370,45.528047


In [3]:
df

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,259172515,259172517,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082,12,0.7
1,259172515,259172517,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070,12,0.8
2,259172515,259172517,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123,12,0.8
3,259172515,259172517,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262,12,0.7
4,259172515,259172517,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582,12,0.8
...,...,...,...,...,...,...,...,...,...,...
156,259172515,259172530,15FEB2023:00:00:00,4223,5834,21389,-122.677057,45.528040,11,0.8
157,259172515,259172531,15FEB2023:00:00:00,4223,5838,21424,-122.677003,45.528037,6,2.0
158,259172515,259172531,15FEB2023:00:00:00,4223,5858,21429,-122.676765,45.528043,7,1.1
159,259172515,259172531,15FEB2023:00:00:00,4223,5889,21434,-122.676370,45.528047,10,1.1


In [4]:
#Filtering using usecols
columns_to_include = [0, 2, 3, 4, 5, 6, 7 ]
df = pd.read_csv("C:/Users/sriva/Downloads/bc_trip259172515_230215.csv", usecols=columns_to_include)

In [5]:
df

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582
...,...,...,...,...,...,...,...
156,259172515,15FEB2023:00:00:00,4223,5834,21389,-122.677057,45.528040
157,259172515,15FEB2023:00:00:00,4223,5838,21424,-122.677003,45.528037
158,259172515,15FEB2023:00:00:00,4223,5858,21429,-122.676765,45.528043
159,259172515,15FEB2023:00:00:00,4223,5889,21434,-122.676370,45.528047


In [6]:
from datetime import datetime, timedelta
def timestamp(row):
    #get the date and time
    opd_date = datetime.strptime(row['OPD_DATE'], '%d%b%Y:%H:%M:%S')
    delta = timedelta(seconds=row['ACT_TIME'])
    return opd_date + delta

df['TIMESTAMP'] = df.apply(timestamp, axis=1)

print(df)

     EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0        259172515  15FEB2023:00:00:00        4223      40     20469   
1        259172515  15FEB2023:00:00:00        4223      48     20474   
2        259172515  15FEB2023:00:00:00        4223      57     20479   
3        259172515  15FEB2023:00:00:00        4223      73     20484   
4        259172515  15FEB2023:00:00:00        4223     112     20489   
..             ...                 ...         ...     ...       ...   
156      259172515  15FEB2023:00:00:00        4223    5834     21389   
157      259172515  15FEB2023:00:00:00        4223    5838     21424   
158      259172515  15FEB2023:00:00:00        4223    5858     21429   
159      259172515  15FEB2023:00:00:00        4223    5889     21434   
160      259172515  15FEB2023:00:00:00        4223    5918     21439   

     GPS_LONGITUDE  GPS_LATITUDE           TIMESTAMP  
0      -122.648137     45.493082 2023-02-15 05:41:09  
1      -122.648240     45

E.  More Filtering

In [7]:
df1=df.drop(['OPD_DATE', 'ACT_TIME'], axis=1)
df1

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29
...,...,...,...,...,...,...
156,259172515,4223,5834,-122.677057,45.528040,2023-02-15 05:56:29
157,259172515,4223,5838,-122.677003,45.528037,2023-02-15 05:57:04
158,259172515,4223,5858,-122.676765,45.528043,2023-02-15 05:57:09
159,259172515,4223,5889,-122.676370,45.528047,2023-02-15 05:57:14


In [8]:
df1['dMETERS'] = df1['METERS'].diff()
df1['dTIMESTAMP'] = df1['TIMESTAMP'].diff()

#Calculating speed
df1['SPEED'] = df1.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'].total_seconds() if row['dTIMESTAMP'].total_seconds() != 0 else 0, axis=1)
print(df1)
# Drop unneeded columns
df2 = df1.drop(['dMETERS', 'dTIMESTAMP'], axis=1)

#Minimum, maximum and average speed
speed = df1['SPEED'].describe()
print(f"\n{df1['SPEED'].describe()}")
print("\nMinimum speed:", speed['min'])
print("Maximum speed:", speed['max'])
print("Average speed:", speed['mean'])

df2

     EVENT_NO_TRIP  VEHICLE_ID  METERS  GPS_LONGITUDE  GPS_LATITUDE  \
0        259172515        4223      40    -122.648137     45.493082   
1        259172515        4223      48    -122.648240     45.493070   
2        259172515        4223      57    -122.648352     45.493123   
3        259172515        4223      73    -122.648385     45.493262   
4        259172515        4223     112    -122.648347     45.493582   
..             ...         ...     ...            ...           ...   
156      259172515        4223    5834    -122.677057     45.528040   
157      259172515        4223    5838    -122.677003     45.528037   
158      259172515        4223    5858    -122.676765     45.528043   
159      259172515        4223    5889    -122.676370     45.528047   
160      259172515        4223    5918    -122.675990     45.528065   

              TIMESTAMP  dMETERS      dTIMESTAMP     SPEED  
0   2023-02-15 05:41:09      NaN             NaT       NaN  
1   2023-02-15 05:41:14  

,EVENT_NO_TRIP,VEHICLE_ID,METERS,GPS_LONGITUDE,GPS_LATITUDE,TIMESTAMP,SPEED
0,259172515,4223,40,-122.648137,45.493082,2023-02-15 05:41:09,NaN
1,259172515,4223,48,-122.648240,45.493070,2023-02-15 05:41:14,1.600000
2,259172515,4223,57,-122.648352,45.493123,2023-02-15 05:41:19,1.800000
3,259172515,4223,73,-122.648385,45.493262,2023-02-15 05:41:24,3.200000
4,259172515,4223,112,-122.648347,45.493582,2023-02-15 05:41:29,7.800000
...,...,...,...,...,...,...,...
156,259172515,4223,5834,-122.677057,45.528040,2023-02-15 05:56:29,2.800000
157,259172515,4223,5838,-122.677003,45.528037,2023-02-15 05:57:04,0.114286
158,259172515,4223,5858,-122.676765,45.528043,2023-02-15 05:57:09,4.000000
159,259172515,4223,5889,-122.676370,45.528047,2023-02-15 05:57:14,6.200000


G. Larger Data Set

In [9]:
#read the dataset
import pandas as pd
df = pd.read_csv("C:/Users/sriva/Downloads/bc_veh4223_230215.csv")
print(df.head()) 

   EVENT_NO_TRIP  EVENT_NO_STOP            OPD_DATE  VEHICLE_ID  METERS  \
0      259173279      259173350  15FEB2023:00:00:00        4223  194493   
1      259173279      259173350  15FEB2023:00:00:00        4223  194525   
2      259173279      259173350  15FEB2023:00:00:00        4223  194562   
3      259173279      259173350  15FEB2023:00:00:00        4223  194598   
4      259173279      259173350  15FEB2023:00:00:00        4223  194628   

   ACT_TIME  GPS_LONGITUDE  GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP  
0     71279    -122.679572     45.517122               8       1.6  
1     71284    -122.679408     45.517392               8       1.9  
2     71290    -122.679227     45.517700               8       2.2  
3     71295    -122.679048     45.517998               9       0.9  
4     71300    -122.678888     45.518255               9       1.3  


In [10]:
#filtering using drop
df.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis=1)
df

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,259173279,259173350,15FEB2023:00:00:00,4223,194493,71279,-122.679572,45.517122,8,1.6
1,259173279,259173350,15FEB2023:00:00:00,4223,194525,71284,-122.679408,45.517392,8,1.9
2,259173279,259173350,15FEB2023:00:00:00,4223,194562,71290,-122.679227,45.517700,8,2.2
3,259173279,259173350,15FEB2023:00:00:00,4223,194598,71295,-122.679048,45.517998,9,0.9
4,259173279,259173350,15FEB2023:00:00:00,4223,194628,71300,-122.678888,45.518255,9,1.3
...,...,...,...,...,...,...,...,...,...,...
7401,259173498,259173583,15FEB2023:00:00:00,4223,251109,84315,-122.641148,45.539688,12,0.8
7402,259173498,259173583,15FEB2023:00:00:00,4223,251162,84320,-122.641138,45.540175,12,0.7
7403,259173498,259173583,15FEB2023:00:00:00,4223,251199,84325,-122.641107,45.540512,12,0.7
7404,259173498,259173583,15FEB2023:00:00:00,4223,251238,84330,-122.641142,45.540860,12,0.7


In [11]:
#Filtering using usecols
columns_to_include = [0, 2, 3, 4, 5, 6, 7 ]
df = pd.read_csv("C:/Users/sriva/Downloads/bc_veh4223_230215.csv", usecols=columns_to_include)
df

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259173279,15FEB2023:00:00:00,4223,194493,71279,-122.679572,45.517122
1,259173279,15FEB2023:00:00:00,4223,194525,71284,-122.679408,45.517392
2,259173279,15FEB2023:00:00:00,4223,194562,71290,-122.679227,45.517700
3,259173279,15FEB2023:00:00:00,4223,194598,71295,-122.679048,45.517998
4,259173279,15FEB2023:00:00:00,4223,194628,71300,-122.678888,45.518255
...,...,...,...,...,...,...,...
7401,259173498,15FEB2023:00:00:00,4223,251109,84315,-122.641148,45.539688
7402,259173498,15FEB2023:00:00:00,4223,251162,84320,-122.641138,45.540175
7403,259173498,15FEB2023:00:00:00,4223,251199,84325,-122.641107,45.540512
7404,259173498,15FEB2023:00:00:00,4223,251238,84330,-122.641142,45.540860


In [12]:
df['OPD_DATE'] = pd.to_datetime(df['OPD_DATE'], format='%d%b%Y:%H:%M:%S')

group = df.groupby('EVENT_NO_TRIP')
dfs = []

for id, data in group:
    data = data.sort_values(by='ACT_TIME')
    
    data['TIMESTAMP'] = data.apply(lambda row: row['OPD_DATE'] + timedelta(seconds=row['ACT_TIME']), axis=1)
    data['dMETERS'] = data['METERS'].diff()
    data['dTIMESTAMP'] = data['TIMESTAMP'].diff()
    
    #Calculate the speed
    data['SPEED'] = data.apply(lambda row: row['dMETERS'] / row['dTIMESTAMP'].total_seconds() if row['dTIMESTAMP'].total_seconds() != 0 else 0, axis=1)
    print(f'\nTrip Number:{id}')
    speed = data['SPEED'].describe()
    print(f"{data['SPEED'].describe()}")
    print("\nMinimum speed:", speed['min'])
    print("Maximum speed:", speed['max'])
    print("Average speed:", speed['mean'])
    dfs.append(data)


df3 = pd.concat(dfs)
#drop the columns
df3 = df3.drop(['OPD_DATE', 'ACT_TIME','dMETERS', 'dTIMESTAMP'], axis=1)
print(df3)




Trip Number:259169000
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: SPEED, dtype: float64

Minimum speed: nan
Maximum speed: nan
Average speed: nan

Trip Number:259172515
count    160.000000
mean       7.227206
std        4.420604
min        0.000000
25%        3.800000
50%        6.400000
75%       10.850000
max       17.400000
Name: SPEED, dtype: float64

Minimum speed: 0.0
Maximum speed: 17.4
Average speed: 7.227205815018314

Trip Number:259172533
count    445.000000
mean       6.934329
std        3.116734
min        0.000000
25%        5.000000
50%        7.400000
75%        9.200000
max       13.400000
Name: SPEED, dtype: float64

Minimum speed: 0.0
Maximum speed: 13.4
Average speed: 6.934328868294691

Trip Number:259172598
count    705.000000
mean       6.713933
std        3.565119
min        0.000000
25%        4.000000
50%        6.800000
75%        9.400000
max       14.600000
Name: SPEED, dtype: float64

Minimum

In [13]:
group2 = df3.groupby('VEHICLE_ID')

speed = []

for id, data in group2:
    # Filter data for '2023-02-15'
    data = data[data['TIMESTAMP'].dt.date == pd.to_datetime('2023-02-15').date()]
    if not data.empty:
        s = data['SPEED'].describe()
        speed.append({
            'VEHICLE_ID': id,
            'MIN_SPEED': s['min'],
            'MAX_SPEED': s['max'],
            'MEDIAN_SPEED': data['SPEED'].median()
        })

df4 = pd.DataFrame(speed)
print(df4)

   VEHICLE_ID  MIN_SPEED  MAX_SPEED  MEDIAN_SPEED
0        4029        NaN        NaN           NaN
1        4223        0.0       17.4           7.2


In [14]:
group3 = df3.groupby('VEHICLE_ID')

max = []

for vehicle_id, vehicle_data in group3:
    if not vehicle_data.empty:
        max_speed = vehicle_data['SPEED'].max()
        max_speed_rows = vehicle_data[vehicle_data['SPEED'] == max_speed].to_dict('records')
        for row in max_speed_rows:
            row['VEHICLE_ID'] = vehicle_id
            max.append(row)

df5 = pd.DataFrame(max)
df5 = df5[df5['TIMESTAMP'].dt.date == pd.to_datetime('2023-02-15').date()]
print(df5)

   EVENT_NO_TRIP  VEHICLE_ID  METERS  GPS_LONGITUDE  GPS_LATITUDE  \
0      259172515        4223    2187    -122.660822     45.505452   

            TIMESTAMP  SPEED  
0 2023-02-15 05:44:49   17.4  
